In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import pandas_datareader as web
from colorama import Fore, Back, Style
import time
import datetime as dt
import os

In [2]:
#package_dir = os.path.dirname(os.path.abspath('FICHIER DU 09 BOOST SAVE'))
#LeChemin = '/'+package_dir+'/20191211 BACKTEST/'
LeChemin = 'DL_2019-12-06/'
start = '2015-12-06'
end = '2020-02-07'
list_dir = os.listdir('DL_2019-12-06/.')
NEW_LIST = []
for pikunichihouahoua in range(0,len(str(list_dir).split(','))):
    if str(list_dir).split(',')[pikunichihouahoua].split(',')[0].split('.')[1].split("'")[0]=='yaml':
        NEW_LIST.append(str(list_dir).split(',')[pikunichihouahoua].split(',')[0].split('.')[0].split("'")[1].split('_')[1])
NEW_LIST = sorted(set(NEW_LIST))

In [3]:
___Author___='LumberJack Jyss'
print('Global Optimized LumberJack Environment Motor 55\nLumberJack Jyss 5780(c)')
print(Fore.BLUE,'°Go!em 73°')
print('ULTIMATE BACKTEST v0.5',Style.RESET_ALL)

Global Optimized LumberJack Environment Motor 55
LumberJack Jyss 5780(c)
 °Go!em 73°
ULTIMATE BACKTEST v0.5 


In [4]:
def scrap_data(ticker,start,end):
    df = web.DataReader(ticker,'yahoo',start,end)
    df = df.drop(['Close'],axis=1)
    df['Close'] = df['Adj Close']
    df = df.drop(['Adj Close'],axis = 1)    
    return(df)

def load_boot(df,model):
    seed = 770
    np.random.seed(seed)
    X = df.copy()
    X = X.drop(['Close'],axis=1)
    X['Close'] = df['Close']
    y = X.iloc[:,-1]
    Xtrain = X.iloc[:-2,:-1]
    Xtest = X.iloc[-2:-1,:-1]
    yshift = y.shift(-1)
    ytrain = yshift.iloc[:-2]
    ytest = yshift.iloc[-2:-1]

    y_pred = model.predict(Xtest)

    pred = model.predict(X.iloc[:,:-1])

    df['Close.S'] = pred
    df['Close.S2'] = df['Close.S']
    df = df.dropna()
    return(df)


def prepa_data(df):
    rsi = talib.RSI(df['Close'],timeperiod=14)
    stoc_slowk, stoc_slowd = talib.STOCH(df['High'],df['Low'],df['Close'])
    upper, middle, lower =  talib.BBANDS(df['Close'], timeperiod=9, nbdevup=2, nbdevdn=2,matype=0)
    sma5 = talib.SMA(df['Close'],timeperiod=5)
    sma8 = talib.SMA(df['Close'],timeperiod=8)
    sma10 = talib.SMA(df['Close'],timeperiod=10)
    sma12 = talib.SMA(df['Close'],timeperiod=12)
    sma15 = talib.SMA(df['Close'],timeperiod=15)
    sma30 = talib.SMA(df['Close'],timeperiod=30)
    sma35 = talib.SMA(df['Close'],timeperiod=35)
    sma40 = talib.SMA(df['Close'],timeperiod=40)
    sma45 = talib.SMA(df['Close'],timeperiod=45)
    sma50 = talib.SMA(df['Close'],timeperiod=50)
    atr = talib.ATR(df['High'],df['Low'],df['Close'],timeperiod=10)
    delta5_8 = sma5 - sma8
    delta8_10 = sma8 - sma10
    delta10_12 = sma10 - sma12
    delta12_15 = sma12 - sma15
    delta15_30 = sma15 - sma30
    delta30_35 = sma30 - sma35
    delta35_40 = sma35 - sma40
    delta40_45 = sma40 - sma45
    delta45_50 = sma45 - sma50
    bbdelta = upper - middle
    price_bolup = df['Close'] - lower
    price_bolow = df['Close'] - upper
    Ema = talib.EMA(df['Close'],timeperiod=20)
    KC_High = Ema + 2*atr
    KC_Low = Ema - 2*atr
    aroondown, aroonup = talib.AROON(df['High'], df['Low'], timeperiod=9)
    aroon = aroonup - aroondown #(aroonup-aroondown)/abs((aroonup-aroondown))
    rsi30_list = []
    rsi70_list = []
    for i in range(0,df.shape[0]):
        rsi70_list.append(70 - rsi[i])
        rsi30_list.append(rsi[i] - 30)
        #except:
         #   rsi70_list.append(0)
          #  rs30_list.append(0)
    varop_spy = df['Open'] - df['Close']
    varhl_spy = df['High'] - df['Low']
    df['Varop_Spy'] = varop_spy
    df['Varhl_spy'] = varhl_spy
    df['RSI'] = rsi
    df['70 - RSI'] = np.array(rsi70_list)
    df['RSI - 30'] = np.array(rsi30_list)
    df['BBD_Delta_Up'] = bbdelta
    df['delta5_8'] = delta5_8
    df['delta8_10'] = delta8_10
    df['delta10_12'] = delta10_12
    df['delta12_15'] = delta12_15
    df['delta15_30'] = delta15_30
    df['delta30_35'] = delta30_35
    df['delta35_40'] = delta35_40
    df['delta40_45'] = delta40_45
    df['delta45_50'] = delta45_50
    df['Stoc_Slowk'] = stoc_slowk
    df['Stoc_Slowd'] = stoc_slowd
    df['KC_High'] = KC_High
    df['KC_Low'] = KC_Low
    df['upper'] = upper
    df['lower'] = lower
    df['var_bollup_kchigh'] = upper-KC_High
    df['var_bolllow_kclow'] = lower-KC_Low
    df['Aroon Up'] = aroonup
    df['Aroon Down'] = aroondown
    df['Delta Aroon'] = aroon
    up = []
    down = []
    df = df.dropna()
    
    df['%Futur'] = ((df['Close.S']-df['Close']) *100) / (df['Close'])
    df['%Futur2'] = ((df['Close.S2']-df['Close']) *100) / (df['Close'])
    for i in range(0,df.shape[0]):
        if df.iloc[i]['%Futur'] > 0.5 :#or df.iloc[i]['%Futur2'] > 0.1:
            up.append(1)
            down.append(0)
        elif df.iloc[i]['%Futur'] < -0.5: #or df.iloc[i]['%Futur2'] < -0.1:
            up.append(0)
            down.append(1)
        else:
            up.append(0)
            down.append(0)

    df['target_up'] = up  # target_up # abs(np.array(valley))#target_up
    df['target_down'] = down # target_down # abs(np.array(peak))#target_down
    df = df[['High','Low','Open','Volume','Close','Varop_Spy','Varhl_spy','RSI','70 - RSI','RSI - 30',\
             'BBD_Delta_Up','delta5_8','delta8_10','delta10_12','delta12_15','delta15_30','delta30_35',\
             'delta35_40','delta40_45','delta45_50','Stoc_Slowk','Stoc_Slowd','KC_High','KC_Low','upper',\
             'lower','var_bollup_kchigh','var_bolllow_kclow','Aroon Up','Aroon Down','Delta Aroon','Close.S',\
             'Close.S2','%Futur','%Futur2','target_up','target_down']]
    #df = df.dropna()
    return(df)

def deep_learning(df):
    seed = 770
    np.random.seed(seed)
    X = df.copy()
    #####################X = X.iloc[:,1:-4]
    X = X.iloc[:,:-4]
    X.astype(np.float64)
    y_up = df.iloc[:,-2].values
    y_down = df.iloc[:,-1].values

    scaler = MinMaxScaler(feature_range=(0, 1))
    X = scaler.fit_transform(X)
    y_up = np.array(y_up).reshape(-1,1)
    y_down = np.array(y_down).reshape(-1,1)

    Xtrain = X[:bloc1,:]
    Xtest = X[bloc1:,:]
    ytrain_up = y_up[:bloc1,:]
    ytest_up = y_up[bloc1:,:]
    ytrain_down = y_down[:bloc1,:]
    ytest_down = y_down[bloc1:,:]

    ytrain_up = ytrain_up.reshape(ytrain_up.shape[0],)
    ytrain_down = ytrain_down.reshape(ytrain_down.shape[0],)

    Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1])

    yhat_up = model_up.predict_classes(Xtest)
    yhat_down = model_down.predict_classes(Xtest)

    predict_up = model_up.predict(Xtest)
    predict_down = model_down.predict(Xtest)

    precision_up = precision_score(ytest_up, yhat_up)  
    precision_down = precision_score(ytest_down, yhat_down) 

    resultats = pd.DataFrame()
    resultats['Date'] = df.index[bloc1:].strftime('%Y-%m-%d')
    resultats.index= df.index[bloc1:]
    resultats['Move Up'] = yhat_up
    resultats['Confiance up'] = (predict_up)*100
    resultats['Move Down'] = yhat_down
    resultats['Confiance Down'] = (predict_down)*100
    resultats['Actual'] = df.iloc[bloc1:]['Close']
    resultats['Actual.S'] = df.iloc[bloc1:]['Close.S']
    open_S = df['Open'].shift(-1)
    resultats['Open.S'] = open_S.iloc[bloc1:]
    dmp_cp=[]
    dmp_cp = ((resultats['Confiance up']-resultats['Confiance Down'])/(resultats['Confiance up']+resultats['Confiance Down'])*100)
    resultats['DMP_CP'] = dmp_cp
    
    return(resultats,precision_up,precision_down,scaler)

In [ ]:
###################################################
########### GENERATION DE BASE ####################
###################################################

print(Fore.YELLOW,'ON GENERE LaBase',Style.RESET_ALL)
tmps55=time.time()
LaBase = pd.DataFrame()
df = pd.DataFrame()
BADTICKER = ['AMCR']
for loop,ticker in enumerate(NEW_LIST):
    try:
        df = web.DataReader(ticker,'yahoo',start,end)
        df['Date'] = df.index
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
    except:
        BADTICKER.append(ticker)
        continue
    df['Symbol'] = ticker
    df = df[['Symbol','High','Low','Open','Close','Volume','Adj Close']]
    LaBase = pd.concat((LaBase,df),ignore_index=False)
tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"Time for completing LaBase Generation = %f" %tmps22,'seconds \n',Style.RESET_ALL)

print('Il y a ',len(BADTICKER),' mauvais tickers')
print('Les voici')
print(BADTICKER)

#LaBase[LaBase.duplicated()].shape

print('Base générée : ',LaBase.shape)
print('Il y a ',LaBase[LaBase.duplicated()].shape,' Duplicats')
print('Hors AMCR, voici les duplicats') 
print(LaBase[(LaBase.duplicated()==True)&(LaBase['Symbol'] != 'AMCR')])

###########################################################
############## SI BESOIN DE RETIRER LA DATE DU JOUR #######
##
#  LaBase = LaBase[LaBase.index != '2019-09-18 00:00:00'] #
##
#
# LaBase.index = pd.to_datetime(LaBase['Date'],format='%Y-%m-%d')
###########################################################
try:
    os.remove('LaBase_17.csv')
except:
    pass
LaBase.to_csv('LaBase_17.csv') 

In [5]:
import talib
from sklearn.preprocessing import MinMaxScaler
import yaml
from keras.models import model_from_yaml
from sklearn.metrics import  precision_score
import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
yaml.warnings({'YAMLLoadWarning': False})
warnings.filterwarnings("ignore")
import pickle

Using TensorFlow backend.


In [7]:
LaBase = pd.read_csv('LaBase_adj.csv')
try:
    LaBase = LaBase.drop(['Unnamed: 0'],axis=1)
except :
    pass
LaBase.index = pd.to_datetime(LaBase['Date'],format='%Y-%m-%d')
try:
    LaBase = LaBase.drop(['Date'],axis=1)
    #LaBase = LaBase.drop(['Close'],axis=1)
    LaBase['Close'] = LaBase['Adj Close']
except:
    pass
try:
    LaBase = LaBase.drop(['Adj Close'],axis=1)
except:
    pass
LaBase,end

(           Symbol        High         Low        Open     Volume       Close
 Date                                                                        
 2015-12-07      A   41.099998   40.540001   40.919998  2004500.0   39.091465
 2015-12-07    AWK   58.410000   57.590000   57.750000   783000.0   53.948914
 2015-12-07   CINF   61.560001   61.060001   61.299999   683500.0   54.486759
 2015-12-07     CL   66.559998   66.089996   66.389999  3895600.0   60.412056
 2015-12-07    IEX   78.620003   77.320000   78.400002   302800.0   73.439034
 ...           ...         ...         ...         ...        ...         ...
 2020-02-07    XYL   84.910004   83.129997   83.360001  2496800.0   84.199997
 2020-02-07    YUM  102.620003  100.750000  101.760002  2396800.0  102.040001
 2020-02-07    ZBH  158.729996  156.119995  158.279999   889300.0  156.389999
 2020-02-07   ZION   46.840000   45.950001   46.840000  2611800.0   46.230000
 2020-02-07    ZTS  139.160004  138.029999  139.050003  1780800.

In [9]:
long = len(NEW_LIST)


BasePrep = pd.read_csv('BasePrep.csv')
try:
    BasePrep = BasePrep.drop(['Unnamed: 0'],axis=1)
except :
    pass
BasePrep.index = pd.to_datetime(BasePrep['Date'],format='%Y-%m-%d')
try:
    BasePrep = BasePrep.drop(['Date'],axis=1)
    #LaBase = LaBase.drop(['Close'],axis=1)
    BasePrep['Close'] = BasePrep['Adj Close']
except:
    pass
try:
    BasePrep = BasePrep.drop(['Adj Close'],axis=1)
except:
    pass
BasePrep = BasePrep[['Symbol','High','Low','Open','Volume','Close',\
                         'Varop_Spy','Varhl_spy','RSI','70 - RSI','RSI - 30',\
                         'BBD_Delta_Up','delta5_8','delta8_10','delta10_12',\
                         'delta12_15','delta15_30','delta30_35','delta35_40',\
                         'delta40_45','delta45_50','Stoc_Slowk','Stoc_Slowd',\
                         'KC_High','KC_Low','upper','lower','var_bollup_kchigh',\
                         'var_bolllow_kclow','Aroon Up','Aroon Down','Delta Aroon',\
                         'Close.S','Close.S2','%Futur','%Futur2','target_up','target_down']]

LaBase = LaBase[(LaBase.index>start)&(LaBase.index<=end)]

BasePrep = BasePrep[(BasePrep.index>start)&(BasePrep.index<=end)]


print(str(BasePrep.index[-1])[:10])

delta = round(LaBase.shape[0])
bloc1 =  round(0.5 * LaBase[LaBase['Symbol']=='A'].shape[0]) #round(delta*0.80)
bloc2 = delta - bloc1
print('Bloc1 = ',bloc1)
filtre_up = 93
filtre_down = 97
filtre_up_close = 80
filtre_down_close = 80

signals = pd.DataFrame(columns = ['Date','Ticker','Open Long','Close Long','Open Short','Close Short','Buy/Sell Price',\
                                  'Quantity','Actual','PNL','Cumulative PNL','Latent PNL'])

backtest = pd.DataFrame(columns = ['Symb.','Name','Sector','Period','Begin.','End',\
                          'Pnl $','Pnl %','Gross Profit','Gross Loss','# Winners','# Loosers',\
                          '% Winners','MaxDD $','MaxDD %','Aver. Win $',\
                          'Aver. Loos $','Aver. Duration',\
                          'Total Trades','% Long','% Short','Profit Factor'])


print(Fore.GREEN,'Initiating Calculation',Style.RESET_ALL)

Resultats = pd.DataFrame()

GREEN = []
RED = []
GREEN_CLOSE = []
RED_CLOSE=[]
TICKER = []
DATE = []
PRICE = []
OPEN_S = []
CONFUP = []
CONFDWN = []
PERUP = []
PERDOWN = []

ERRORDATE = []
ERRORLONG = []

tmps55=time.time()
for loop,ticker in enumerate(NEW_LIST):
    
    if loop == 1:
            print('\r',Fore.GREEN,' |==',Fore.BLUE,'O',Fore.RED,'==================|',end='',flush=True)
    if loop == round(1 * long / 9):
        print('\r',Fore.GREEN,' |====',Fore.BLUE,'O',Fore.RED,'================|',end='',flush=True)
    if loop == round(2 * long / 9):
        print('\r',Fore.GREEN,' |======',Fore.BLUE,'O',Fore.RED,'==============|',end='',flush=True)
    if loop == round(3 * long / 9):
        print('\r',Fore.GREEN,' |========',Fore.BLUE,'O',Fore.RED,'============|',end='',flush=True)
    if loop == round(4 * long / 9):
        print('\r',Fore.GREEN,' |==========',Fore.BLUE,'O',Fore.RED,'==========|',end='',flush=True)
    if loop == round(5 * long / 9):
        print('\r',Fore.GREEN,' |============',Fore.BLUE,'O',Fore.RED,'========|',end='',flush=True)
    if loop == round(6 * long / 9):
        print('\r',Fore.GREEN,' |==============',Fore.BLUE,'O',Fore.RED,'======|',end='',flush=True)
    if loop == round(7 * long / 9):
        print('\r',Fore.GREEN,' |================',Fore.BLUE,'O',Fore.RED,'====|',end='',flush=True)
    if loop == round(8 * long / 9):
        print('\r',Fore.GREEN,' |==================',Fore.BLUE,'O',Fore.RED,'==|',Style.RESET_ALL,end='',flush=True)
    
    
    LeChemin = 'DL_2019-12-06/'
    yamlup = (LeChemin+'Save_'+ticker+'_up.yaml')
    yamldown = (LeChemin+'Save_'+ticker+'_down.yaml')
    modelup = (LeChemin+'Save_'+ticker+'_up.h5')
    modeldown = (LeChemin+'Save_'+ticker+'_down.h5')

    yaml_file_up = open(yamlup, 'r')
    yaml_file_down = open(yamldown, 'r')
    model_yaml_up = yaml_file_up.read()
    model_yaml_down = yaml_file_down.read()
    yaml_file_up.close()
    yaml_file_down.close()
    model_up = model_from_yaml(model_yaml_up)
    model_down = model_from_yaml(model_yaml_down)
    
    # load weights into new model
    model_up.load_weights(modelup)
    model_down.load_weights(modeldown)
        
    df = BasePrep[BasePrep['Symbol']==ticker]
    if df.shape[0] < 70:
        ERRORLONG.append(ticker)
        continue
    elif str(df.index[-1])[:10] != '2020-02-07':
        ERRORDATE.append(ticker)
        continue
    
    df = df.drop(['Symbol'],axis=1)
                 
    df = df[['High','Low','Open','Volume','Close','Varop_Spy',\
             'Varhl_spy','RSI','70 - RSI','RSI - 30','BBD_Delta_Up',\
             'delta5_8','delta8_10','delta10_12','delta12_15','delta15_30',\
             'delta30_35','delta35_40','delta40_45','delta45_50',\
             'Stoc_Slowk','Stoc_Slowd','KC_High','KC_Low','upper',\
             'lower','var_bollup_kchigh','var_bolllow_kclow','Aroon Up',\
             'Aroon Down','Delta Aroon','Close.S','Close.S2','%Futur','%Futur2','target_up','target_down']]
    
    df.to_csv('PrepData/prepdata_'+ticker+'_09.csv')
    
    resultats,precision_up,precision_down,scaler = deep_learning(df)
    
    
    #if len(resultats) < 252:
     #   error_2.append(ticker)
      #  continue
    
    if (precision_up * 100) > 87 and (precision_down * 100) > 87:

        ####################
        ##### SIGNALS #####
        ###################

        filtre_up = 93
        filtre_down = 97
        filtre_up_close = 80
        filtre_down_close = 80

        for i in range(0,len(resultats)):

            price = resultats.iloc[i]['Actual']

            if resultats.iloc[i]['Confiance Down'] > filtre_down:
                doob = -1
            elif resultats.iloc[i]['Confiance Down'] > filtre_down_close and resultats.iloc[i]['Confiance Down'] < filtre_down :
                waab = -1
                doob = 0
            elif resultats.iloc[i]['Confiance up'] > filtre_up:
                doob = 1
            elif resultats.iloc[i]['Confiance up'] > filtre_up_close and resultats.iloc[i]['Confiance up'] < filtre_up:
                waab = 1
                doob = 0
            else :
                doob = 0
                waab = 0

            if doob == 1 :
                DATE.append(resultats.iloc[i]['Date'])
                TICKER.append(ticker)
                GREEN.append(1)
                RED.append(0)
                GREEN_CLOSE.append(0)
                RED_CLOSE.append(1)
                PRICE.append(price)
                OPEN_S.append(resultats.iloc[i]['Open.S'])
                CONFUP.append(resultats.iloc[i]['Confiance up'])
                CONFDWN.append(resultats.iloc[i]['Confiance Down'])
                PERUP.append(round(precision_up*100,2))
                PERDOWN.append(round(precision_down*100,2))
                

            elif doob == -1 :
                DATE.append(resultats.iloc[i]['Date'])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(1)
                GREEN_CLOSE.append(1)
                RED_CLOSE.append(0)
                PRICE.append(price)
                OPEN_S.append(resultats.iloc[i]['Open.S'])
                CONFUP.append(resultats.iloc[i]['Confiance up'])
                CONFDWN.append(resultats.iloc[i]['Confiance Down'])
                PERUP.append(round(precision_up*100,2))
                PERDOWN.append(round(precision_down*100,2))
                

            elif doob == 0 and waab == -1 :
                DATE.append(resultats.iloc[i]['Date'])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(0)
                GREEN_CLOSE.append(1)
                RED_CLOSE.append(0)
                PRICE.append(price)
                OPEN_S.append(resultats.iloc[i]['Open.S'])
                CONFUP.append(resultats.iloc[i]['Confiance up'])
                CONFDWN.append(resultats.iloc[i]['Confiance Down'])
                PERUP.append(round(precision_up*100,2))
                PERDOWN.append(round(precision_down*100,2))
                

            elif doob == 0 and waab == 1 :
                DATE.append(resultats.iloc[i]['Date'])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(0)
                GREEN_CLOSE.append(0)
                RED_CLOSE.append(1)
                PRICE.append(price)
                OPEN_S.append(resultats.iloc[i]['Open.S'])
                CONFUP.append(resultats.iloc[i]['Confiance up'])
                CONFDWN.append(resultats.iloc[i]['Confiance Down'])
                PERUP.append(round(precision_up*100,2))
                PERDOWN.append(round(precision_down*100,2))
                

            else:
                DATE.append(resultats.iloc[i]['Date'])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(0)
                GREEN_CLOSE.append(0)
                RED_CLOSE.append(0)
                PRICE.append(price)
                OPEN_S.append(resultats.iloc[i]['Open.S'])
                CONFUP.append(resultats.iloc[i]['Confiance up'])
                CONFDWN.append(resultats.iloc[i]['Confiance Down'])
                PERUP.append(round(precision_up*100,2))
                PERDOWN.append(round(precision_down*100,2))
                
                
    '''else:
        DATE.append(resultats.iloc[i]['Date'])
        TICKER.append(ticker)
        GREEN.append(0)
        RED.append(0)
        GREEN_CLOSE.append(0)
        RED_CLOSE.append(0)
        PRICE.append(price)
        OPEN_S.append(resultats.iloc[i]['Open.S'])
        CONFUP.append(resultats.iloc[i]['Confiance up'])
        CONFDWN.append(resultats.iloc[i]['Confiance Down'])
        '''

Resultats['Date'] = DATE
Resultats['Ticker'] = TICKER
Resultats['Conf Up'] = CONFUP
Resultats['Conf Dwn'] = CONFDWN
Resultats['Green'] = GREEN
Resultats['Red'] = RED
Resultats['Green_Close'] = GREEN_CLOSE
Resultats['Red_Close'] = RED_CLOSE
Resultats['Price'] = PRICE
Resultats['Open.S'] = OPEN_S
Resultats['% up'] = PERUP
Resultats['% down'] = PERDOWN

tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"\n Time for completing Signal Generation = %f" %tmps22,'seconds \n',Style.RESET_ALL)
Resultats = Resultats[(Resultats['Green_Close']==1)|(Resultats['Red_Close']==1)]
Resultats.to_csv('Resultats_live.csv')


2020-02-07
Bloc1 =  524
 Initiating Calculation 
   |==================  O  ==|  
 Time for completing Signal Generation = 172.130000 seconds 
 


In [10]:
len(ERRORDATE),len(ERRORLONG)

(5, 46)

In [11]:
loop,ticker,sum(GREEN),sum(GREEN_CLOSE),sum(RED),sum(RED_CLOSE),len(GREEN),len(set(TICKER)),DATE[-1]

(342, 'ZION', 2408, 4521, 3835, 2812, 12342, 26, '2020-02-07')

In [12]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import sys
import codecs
%matplotlib inline
plt.style.use('seaborn')
orig_stdout = sys.stdout

In [13]:
########################
####### BACKTEST########
########################
LeChemin=''
resultats = Resultats
resultats['Date'] = pd.to_datetime(resultats['Date'],format='%Y-%m-%d')
tmps55=time.time()
error_3 = []
for ticker in NEW_LIST:
    if ticker in ERRORLONG or ticker in ERRORDATE:
        continue
    pos_long = 0
    pos_short = 0
    pactol = 1000000
    mise = 30000
    price_buy = 0
    price_sell = 0
    x = 0
    winners = []
    loosers = []
    nb_win = 0
    nb_los = 0
    mini_pnl = 0
    pos_duration = []
    average_duration = [] # pd.Timedelta(resultats.iloc[0,0] - resultats.iloc[0,0])
    trades = []
    eq_curx = []
    eq_cury = []
    maxdd = []
    nbS = 0
    nbL = 0
    
    
    f = codecs.open('BT/backtest de '+str(ticker)+'.txt', 'w','utf-8')
    f.write(u'\ufeff')
    sys.stdout = f
    print('TICKER : ',ticker)
    resultats = Resultats[Resultats['Ticker'] == ticker]
    if resultats.shape[0] < 1:
        error_3.append(ticker)
        continue
    for i in range(0,len(resultats)-1):
        if pos_long == 0 and pos_short == 0 and resultats.iloc[i]['Green'] == 1 :
            pos_long = 1
            nbL+=1
            x = round((mise)/resultats.iloc[i]['Open.S'])
            price_buy = round(resultats.iloc[i]['Open.S'],2)
            time_enter = resultats.iloc[i]['Date']

            signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 1,\
                               'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 0,\
                               'Buy/Sell Price' : price_buy, 'Quantity' : x,'Actual' : round(resultats.iloc[i]['Price'],2), 'PNL' : 0,\
                                       'Cumulative PNL':round(signals.iloc[:i]['PNL'].sum(),2),\
                                       'Latent PNL':0}],ignore_index=True)

            print('\n Le ',resultats.iloc[i]['Date'].strftime('%Y-%m-%d'),', achat de ',x,' actions à',round(resultats.iloc[i]['Open.S'],2),'$')
            print('Nouveau pactol : ',round(pactol,2))
            print('Le Cumulative PNL est de : ',round(signals.iloc[:i]['PNL'].sum(),2))
            print('Le Latent PNL est de : 0')
            trades.append((round(resultats.iloc[i]['Open.S'],2),resultats.iloc[i]['Date'],1))

        elif pos_long == 1 and resultats.iloc[i]['Green_Close'] == 0 :

            signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                               'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 0,\
                               'Buy/Sell Price' : price_buy, 'Quantity' : x,'Actual' : round(resultats.iloc[i]['Price'],2), 'PNL' : 0,\
                                       'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),\
                                       'Latent PNL':round((resultats.iloc[i]['Price'] - price_buy)*x,2)}],ignore_index=True)
            try:
                maxdd.append(round((resultats.iloc[i]['Open.S'] - price_buy)*x,2))
            except:
                print('Null val of maxdd')
                maxdd.append(0)
                print('\n Le ',str(resultats.iloc[i]['Date']))
                print('Nouveau pactol : ',round(pactol,2))
                print('Le Cumulative PNL est de : ',round(signals.iloc[:i]['PNL'].sum(),2))
                print('Le Latent PNL est de : ',maxdd[-1])

        elif pos_long == 1 and resultats.iloc[i]['Green_Close'] == 1 :
            old_pactol = round(pactol,2)
            eq_curx.append(resultats.iloc[i]['Date'])
            eq_cury.append(old_pactol)
            pos_long = 0
            price_sell = round(resultats.iloc[i]['Open.S'],2)
            mini_pnl = round(x*(price_sell-price_buy),2)
            trades.append((round(resultats.iloc[i]['Open.S'],2),resultats.iloc[i]['Date'],2))

            signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                               'Close Long' : 1, 'Open Short' : 0, 'Close Short' : 0,\
                               'Buy/Sell Price' : price_sell, 'Quantity' : x,'Actual' : round(resultats.iloc[i]['Price'],2), 'PNL' : mini_pnl,\
                                       'Cumulative PNL':round(signals.iloc[:i]['PNL'].sum(),2),'Latent PNL':0}],ignore_index=True)

            pactol = round((pactol + mini_pnl),2)
            pos_duration.append(resultats.iloc[i]['Date'] - time_enter)
            print('\n Le ',str(resultats.iloc[i]['Date']),', vente pour close de ',x,' actions à',round(resultats.iloc[i]['Open.S'],2),'$')
            print('Nouveau pactol : ',pactol)
            print('pnl:', mini_pnl)
            print('prix de vente',price_sell)
            print('prix d achat',price_buy)
            print('Le Cumulative PNL est de : ',round(signals.iloc[:i]['PNL'].sum(),2))
            print('Le Latent PNL est de : 0')
            print('Duration : ',pos_duration[-1])
            

            if mini_pnl > 0:
                winners.append(mini_pnl)
                mini_pnl = 0
                nb_win +=1
            else :
                loosers.append(mini_pnl)
                mini_pnl = 0
                nb_los +=1


        elif pos_short == 0 and pos_long == 0 and resultats.iloc[i]['Red']==1:
            pos_short = 1
            nbS+=1
            x = round((mise)/resultats.iloc[i]['Open.S'])
            price_sell = round(resultats.iloc[i]['Open.S'],2)
            time_enter = resultats.iloc[i]['Date']

            signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                               'Close Long' : 0, 'Open Short' : 1, 'Close Short' : 0,\
                               'Buy/Sell Price' : price_sell, 'Quantity' : x,'Actual' : round(resultats.iloc[i]['Price'],2), 'PNL' : 0,\
                                       'Cumulative PNL':round(signals.iloc[:i]['PNL'].sum(),2),'Latent PNL':0}],ignore_index=True)

            print('\n Le ',str(resultats.iloc[i]['Date']),', vente de ',x,' actions à',round(resultats.iloc[i]['Open.S'],2),'$')
            print('Nouveau pactol : ',pactol)
            print('Le Cumulative PNL est de : ',round(signals.iloc[:i]['PNL'].sum(),2))
            print('Le Latent PNL est de : 0')
            
            trades.append((round(resultats.iloc[i]['Open.S'],2),resultats.iloc[i]['Date'],-1))


        elif pos_short == 1 and resultats.iloc[i]['Red_Close'] == 0 :

            signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                               'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 0,\
                               'Buy/Sell Price' : price_sell, 'Quantity' : x,'Actual' : round(resultats.iloc[i]['Price'],2), 'PNL' : 0,\
                                       'Cumulative PNL':round(signals.iloc[:i]['PNL'].sum(),2),\
                                       'Latent PNL':-round((resultats.iloc[i]['Price'] - price_sell)*x,2)}],ignore_index=True)
            try:
                maxdd.append(-round((resultats.iloc[i]['Open.S'] - price_sell)*x,2))
            except:
                print('Null val of maxdd')
                maxdd.append(0)
                print('\n Le ',str(resultats.iloc[i]['Date']))
                print('Nouveau pactol : ',round(pactol,2))
                print('Le Cumulative PNL est de : ',round(signals.iloc[:i]['PNL'].sum(),2))
                print('Le Latent PNL est de : ',maxdd[-1])

        elif pos_short == 1 and resultats.iloc[i]['Red_Close'] == 1 :
            old_pactol = round(pactol,2)
            eq_curx.append(resultats.iloc[i]['Date'])
            eq_cury.append(old_pactol)
            pos_short = 0
            price_buy = round(resultats.iloc[i]['Open.S'],2)
            trades.append((round(resultats.iloc[i]['Open.S'],2),resultats.iloc[i]['Date'],-2))
            mini_pnl = round(x*(price_sell-price_buy),2)

            signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                               'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 1,\
                               'Buy/Sell Price' : price_buy,'Quantity' : x,'Actual' : round(resultats.iloc[i]['Price'],2), 'PNL' : mini_pnl,\
                                       'Cumulative PNL':round(signals.iloc[:i]['PNL'].sum(),2),'Latent PNL':0}],ignore_index=True)

            pactol = round((pactol + mini_pnl),2)
            print('\n Le ',str(resultats.iloc[i]['Date']),', achat pour close de ',x,' actions à',round(resultats.iloc[i]['Open.S'],2),'$')
            print('Nouveau pactol : ',pactol)

            print('prix de vente',price_sell)
            print('prix d achat',price_buy)

            print('pnl:', mini_pnl)
            pos_duration.append(resultats.iloc[i]['Date'] - time_enter)
            
            print('Le Cumulative PNL est de : ',round(signals.iloc[:i]['PNL'].sum(),2))
            print('Le Latent PNL est de : 0')
            print('Duration : ',pos_duration[-1])
            
            if mini_pnl > 0:
                winners.append(mini_pnl)
                mini_pnl = 0
                nb_win +=1
            else :
                loosers.append(mini_pnl)
                mini_pnl = 0
                nb_los +=1
        else:

            signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                           'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 0,\
                           'Buy/Sell Price' : 0, 'Quantity' : 0,'Actual' : round(resultats.iloc[i]['Price'],2), 'PNL' : 0,\
                                       'Cumulative PNL':round(signals.iloc[:i]['PNL'].sum(),2),'Latent PNL':0}],ignore_index=True)

    if pos_long == 1:
        pos_long = 0
        print('Cutting unresolved position')
        print(' Le price buy était de : ',price_buy)
        x = round((mise/resultats.iloc[i]['Open.S']))
        print('x = ',x)
        price_sell = round(resultats.iloc[i]['Open.S'],2)
        print('Et le price sell : ',price_sell)
        mini_pnl = round(x*(price_sell-price_buy),2)

        signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                               'Close Long' : 1, 'Open Short' : 0, 'Close Short' : 0,\
                               'Buy/Sell Price' : price_sell, 'Quantity' : x,'Actual' : round(resultats.iloc[i]['Price'],2), 'PNL' : mini_pnl,\
                                   'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),'Latent PNL':0}],ignore_index=True)

        pactol = round((pactol + mini_pnl),2)
        print('pnl : ',mini_pnl)
        if mini_pnl > 0:
            winners.append(mini_pnl)
            mini_pnl = 0
            nb_win +=1
        else :
            loosers.append(mini_pnl)
            mini_pnl = 0
            nb_los +=1


    if pos_short == 1:
        pos_short = 0
        print('Cutting uresolved position')
        print(' Le price sell était de : ',price_sell)
        x = round((mise)/resultats.iloc[i]['Open.S'])
        print('x = ',x)
        price_buy = round(resultats.iloc[i]['Open.S'],2)
        print('Et le price_buy : ',price_buy)
        mini_pnl = round(x*(price_sell-price_buy),2)

        signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                               'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 1,\
                               'Buy/Sell Price' : price_buy, 'Quantity' : x,'Actual' : round(resultats.iloc[i]['Price'],2), 'PNL' : mini_pnl,\
                                   'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),'Latent PNL':0}],ignore_index=True)

        pactol = round((pactol + mini_pnl),2)
        print('pnl : ',mini_pnl)

        if mini_pnl > 0:
            winners.append(mini_pnl)
            mini_pnl = 0
            nb_win +=1
        else :
            loosers.append(mini_pnl)
            mini_pnl = 0
            nb_los +=1


    pnl = round(pactol - (1000000),2)
    print('\n Begining of BackTest :',resultats.iloc[0]['Date'])
    print('Instrument :',ticker)
    print('Face value per trade : $',mise)
    print('End of BackTest :',resultats.iloc[-1]['Date'])
    duration = pd.to_timedelta((resultats.iloc[-1]['Date'] - resultats.iloc[0]['Date']),unit='d')
    print("BackTest's period :",duration)
    print('Pnl :',round(pnl,2),'$')
    print('% Pnl : ',round(pnl/(mise)*100,2))
    if pnl == 0:
        continue
    print('Total winners :',round(sum(winners),2),'$')
    print('Total loosers :',round(sum(loosers),2),'$')
    print('Number of winners :',round(nb_win))
    print('Number of loosers :',round(nb_los))
    if nb_win == 0:
        nb_win += 0.00001
    if nb_los == 0:
        nb_los += 0.00001
    print('Maximum Drawdown : ',round(min(maxdd),2))
    print('% Max Drawdown : ',round(min(maxdd)/(mise)*100,2),'%')
    print('Nombre toatl de trades : ',round(nbL+nbS,2))
    print('% de trades Long : ',round((nbL*100)/(nbL+nbS),2))
    print('% de trades Short : ',round((nbS*100)/(nbL+nbS),2))
    try:
        print('% winners :', round((nb_win/(nb_win+nb_los) * 100),2),'%')
        print('Average winners :',(round(sum(winners)/(nb_win))))
    except:
        print('% winners : null')
    try:
        print('Average losers :',(round(sum(loosers)/(nb_los))))
    except:
        print('Average losers : Inf')
    try:
        for t in range(0,len(pos_duration)):
            average_duration = pd.to_timedelta((average_duration + pos_duration[t]),unit='d')
        average_duration = average_duration/(nb_los+nb_win)
        print('Average Duration :', average_duration)
    except:
        print('No Average Duration available')
    try:
        print('Profit Factor : ',abs(round((sum(winners)/sum(loosers)),2)))
    except:
        print('Profit Factor : null')
        
    

    ########################
    ##### PLOT TRADES ######
    ########################

    #plt.figure(figsize=(26,8),dpi=300)
    #plt.title('Deep Learning '+ticker+' Trades')
    #plt.plot(resultats['Open.S'],color='blue',lw=0.9)
    #for i in range(0,len(trades)):
    #    if trades[i][2] == 1:
    #        plt.scatter(x=trades[i][1],y=trades[i][0],c='green',marker='^',lw=2.5)
    #    elif trades[i][2] == -1:
    #        plt.scatter(x=trades[i][1],y=trades[i][0],c='red',marker='^',lw=2.5)
    #    elif trades[i][2] == 2:
    #        plt.scatter(x=trades[i][1],y=trades[i][0],c='g',marker='x',lw=2.5)
    #    elif trades[i][2] == -2:
    #        plt.scatter(x=trades[i][1],y=trades[i][0],c='r',marker='x',lw=2.5)

    #plt.scatter(x=resultats.iloc[i]['Date'],y=resultats.iloc[i]['Open.S'],c='g',marker='^',label='Open Long')
    #plt.scatter(x=resultats.iloc[i]['Date'],y=resultats.iloc[i]['Open.S'],c='r',marker='^',label='Open Short')
    #plt.scatter(x=resultats.iloc[i]['Date'],y=resultats.iloc[i]['Open.S'],c='g',marker='x',label='Close Long')
    #plt.scatter(x=resultats.iloc[i]['Date'],y=resultats.iloc[i]['Open.S'],c='r',marker='x',label='Close Short')
    
    #plt.legend()
    #plt.savefig('backtest de '+str(ticker)+'_02.pdf')
    #plt.close()

    #########################
    ##### PLOT E-CURVE ######
    #########################
    plt.figure(figsize=(26,8),dpi=300)
    plt.title('Deep Learning '+ticker+' Equity Curve')
    plt.plot(eq_curx,eq_cury)
    plt.scatter(x=eq_curx,y=eq_cury,c='orange',marker='o',lw=2.5)
    plt.savefig('BT/backtest de '+str(ticker)+'_e_curve.pdf')
    plt.close()

    
    
    #print('\r Boucle ', loop, 'terminée...',end='')
    #print('Opération restantes :', len(constituents)-loop,'/',len(constituents),'\n')

    name = 'Name'
    sector = 'Sector'
    backtest = backtest.append([{'Symb.':ticker,'Name':name,'Sector':sector,'Period':average_duration,'Begin.':resultats.iloc[0,0],'End':resultats.iloc[-1,0],\
            'Pnl $':pnl,'Pnl %':round(pnl/(mise)*100,2),'Gross Profit':round(sum(winners),2),'Gross Loss':round(sum(loosers),2),'# Winners':nb_win,'# Loosers':nb_los,\
            '% Winners':round((nb_win/(nb_win+nb_los) * 100),2),'MaxDD $':round(min(maxdd),2),'MaxDD %':round(min(maxdd)/(mise)*100,2),'Aver. Win $':(round(sum(winners)/(nb_win+0.0001))),\
            'Aver. Loos $':(round(sum(loosers)/(nb_los+0.0001))),'Aver. Duration':average_duration,\
            'Total Trades':nbL+nbS,'% Long':round((nbL*100)/(nbL+nbS),2),'% Short':round((nbS*100)/(nbL+nbS),2), 'Profit Factor':abs(round((sum(winners)/sum(loosers)),2))}], ignore_index=True)


    #signals = signals.append([{ 'Date': list_signals[0], 'Ticker' : list_signals[8], 'Open Long' : list_signals[1],\
                               #'Close Long' : list_signals[2], 'Open Short' : list_signals[3], 'Close Short' : list_signals[4],\
                               #'Buy/Sell Price' : list_signals[5], 'Quantity' : list_signals[6], ' tPNL' : list_signals[7]}])

    #signals['Date'] = list_signals[0]
    #signals['Ticker'] = list_signals[8]
    #signals['Open Long'] = list_signals[1]
    #signals['Close Long'] = list_signals[2]
    #signals['Open Short'] = list_signals[3]
    #signals['Close Short'] = list_signals[4]
    #signals['Buy/Sell Price'] = list_signals[5]
    #signals['Quantity'] = list_signals[6]
    #signals['tPNL'] = list_signals[7]

    # symbol / name / sector / period of backtest / debut / fin / pnl / total winners / total loosers / Nb winners / nb loosers / 
    # / % winners / Max DD / Aver Win / Aver Loos / Average duration / Profit Factor
f.close()

sys.stdout = orig_stdout
print('Done.')
len(error_3)

Done.


266

In [16]:
f.close()
sys.stdout = orig_stdout
#sys.stdout = sys.__stdout__

In [19]:
print(Fore.YELLOW,'Writing BACKTEST CSV AND SIGNALS.CSV',Style.RESET_ALL)
backtest.to_csv('BackTest_live.csv')
signals = signals.sort_values(by=['Date'])
try:
    signals = signals.drop(['Cumulative PNL'],axis=1)
except:
    pass
signals = signals.reset_index(drop=True)
#signals = signals[(signals['Close Long']==1)|(signals['Close Short']==1)]
signals.to_csv('SIGNALS_live.csv')

tmps22=round(time.time()-tmps55,2)
print(Fore.BLUE,"\n Time for completing BackTest Generation = %f" %tmps22,'seconds \n',Style.RESET_ALL)

print(Fore.CYAN,'BACKTEST CSV WRITED AND SIGNALS.CSV',Style.RESET_ALL)
    #except:
     #   print(Fore.RED,'Problème loop : ',loop,Style.RESET_ALL)
      #  error.append((loop,ticker))

        #continue

 Writing BACKTEST CSV AND SIGNALS.CSV 
 
 Time for completing BackTest Generation = 437.900000 seconds 
 
 BACKTEST CSV WRITED AND SIGNALS.CSV 


In [ ]:
i,nb_los,nb_win,nbL,nbS,ticker,resultats.shape

In [ ]:
len(error_3)

In [ ]:
CUMULGLOB = []
for dujour in sorted(signals['Date'].unique())[:-1]:
    cumul = signals[signals['Date']==dujour]['PNL'].sum()
    CUMULGLOB.append(cumul)

In [ ]:
signals

In [ ]:
GLOURF = 0
for pezzomandiataforri in NEW_LIST:
    
    if BasePrep[BasePrep['Symbol']==pezzomandiataforri].shape[0] == 0:
        GLOURF += 1
    elif BasePrep[BasePrep['Symbol']==pezzomandiataforri].index[-1] != '2020-01-03':
        GLOURF += 1
GLOURF

In [ ]:
BLAFOUMOUK = []

for mazzallerilititiccimati in NEW_LIST:
    try:
        BLAFOUMOUK.append((mazzallerilititiccimati,BasePrep[BasePrep['Symbol']==mazzallerilititiccimati].index[-1]))
    except:
        BLAFOUMOUK.append((mazzallerilititiccimati,'NONE-RIEN-NADA-NICHTS-WALLOU'))

Brilkkhästen = 0
for draltinsgöw in range(0,len(BLAFOUMOUK)):
    if str(BLAFOUMOUK[draltinsgöw][1])[:10] != '2020-01-03':
        Brilkkhästen += 1
        print (BLAFOUMOUK[draltinsgöw][0],BLAFOUMOUK[draltinsgöw][1])
print('Total des chpunz :',Brilkkhästen)

In [ ]:
noulounoulou = signals
noulounoulou = noulounoulou[(noulounoulou['Close Long']==1)|(noulounoulou['Close Short']==1)|(noulounoulou['Open Long']==1)|(noulounoulou['Open Short']==1)]
noulounoulou.index= noulounoulou['Date']
noulounoulou = noulounoulou.drop(['Date'],axis=1)
noulounoulou = noulounoulou.iloc[:,:5]
noulounoulou = noulounoulou[noulounoulou.index=='2019-12-26']
noulounoulou = noulounoulou.sort_values(['Ticker'])
tibidoubida = pd.read_csv('SIGNALS_2020-01-03.csv')
tibidoubida.index = tibidoubida['Date']
tibidoubida = tibidoubida.drop(['Unnamed: 0'],axis=1)
tibidoubida = tibidoubida.drop(['Date'],axis=1)
tibidoubida = tibidoubida.iloc[:,:-1]
tibidoubida = tibidoubida.sort_values(['Ticker'])

In [ ]:
tibidoubida.shape, noulounoulou.shape

In [ ]:
noulounoulou.merge(tibidoubida, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only'].shape

In [ ]:
noulounoulou.merge(tibidoubida, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='right_only'].shape

In [ ]:
noulounoulou[(noulounoulou['Open Long']==1)|(noulounoulou['Open Short']==1)]

In [ ]:
lksdjfhqk = Resultats[Resultats['Date']=='2020-01-03']

In [ ]:
lksdjfhqk.index=lksdjfhqk.Date
lksdjfhqk=lksdjfhqk.drop(['Date'],axis=1)
lksdjfhqk=lksdjfhqk.drop(['Conf Up'],axis=1)
lksdjfhqk=lksdjfhqk.drop(['Conf Dwn'],axis=1)
lksdjfhqk=lksdjfhqk.iloc[:,:-2]
lksdjfhqk=lksdjfhqk.dropna()
tibidoubida=tibidoubida.dropna()
lksdjfhqk.equals(tibidoubida)

In [ ]:
lksdjfhqk.to_csv('PlïklôChvinsk Prhakk.csv')

In [ ]:
lksdjfhqk.merge(tibidoubida, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']

In [ ]:
lksdjfhqk.merge(tibidoubida, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='right_only']

In [ ]:
lksdjfhqk.head()

In [ ]:
tibidoubida.head()

In [14]:
Resultats[Resultats['Date']=='2020-02-07']

,Date,Ticker,Conf Up,Conf Dwn,Green,Red,Green_Close,Red_Close,Price,Open.S,% up,% down
945,2020-02-07,AGN,0.000026,99.939774,0,1,1,0,197.050003,NaN,87.60,94.07
2366,2020-02-07,CPRI,99.947372,0.046831,1,0,0,1,30.500000,NaN,96.18,90.97
2842,2020-02-07,CSX,0.000183,99.999985,0,1,1,0,77.870003,NaN,90.53,99.57
3318,2020-02-07,CVX,0.000000,100.000000,0,1,1,0,108.940002,NaN,93.81,93.33
4270,2020-02-07,EMR,0.000000,100.000000,0,1,1,0,73.769997,NaN,98.77,92.55
5222,2020-02-07,GILD,0.000242,99.999985,0,1,1,0,68.870003,NaN,97.93,98.24
5698,2020-02-07,HSIC,0.000000,99.999962,0,1,1,0,69.970001,NaN,89.68,96.64
6171,2020-02-07,INCY,98.860161,0.231073,1,0,0,1,74.139999,NaN,94.65,87.12
7122,2020-02-07,JEF,0.000000,100.000000,0,1,1,0,23.070000,NaN,88.76,88.44
7598,2020-02-07,JNPR,0.000000,100.000000,0,1,1,0,23.760000,NaN,93.51,97.78


In [18]:
signals[signals['Date']=='2020-02-06']

,Date,Ticker,Open Long,Close Long,Open Short,Close Short,Buy/Sell Price,Quantity,Actual,PNL,Latent PNL
7315,2020-02-06,EMR,0,0,0,1,74.94,400,75.25,636,0
7316,2020-02-06,GILD,0,0,0,0,67.29,446,68.21,0,-410.32
7317,2020-02-06,GILD,0,0,0,1,69.98,429,68.21,-1154.01,0
7318,2020-02-06,HSIC,0,0,0,0,69.56,431,70.97,0,-607.71
7319,2020-02-06,HSIC,0,0,0,1,70.5,426,70.97,-400.44,0
7320,2020-02-06,CPRI,0,1,0,0,30.89,971,31.34,-3301.4,0
7325,2020-02-06,WRK,0,0,0,0,37.03,810,41.67,0,-3758.4
7322,2020-02-06,INCY,0,0,0,0,81.08,370,75.81,0,-1949.9
7323,2020-02-06,INCY,0,1,0,0,75.66,397,75.81,-2151.74,0
7324,2020-02-06,WRK,0,0,0,1,41.31,726,41.67,-3107.28,0
